In [1]:
!pip install geopandas
!pip install osmnx
!pip install scipy
!pip installn networkx!pip install geopandas
!pip install osmnx
!pip install scipy
!pip installn networkx

^C


ERROR: unknown command "installn" - maybe you meant "install"



ERROR: unknown command "installn" - maybe you meant "install"



In [1]:
import pickle

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

Analisi istruzione no pesi


In [8]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_BIBLIOTECHE_CLEAN),
     "tipo": "Biblioteche",
     "gdf":  gpd.read_file(PATH_SCUOLE_CLEAN),
     "tipo": "Scuole"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, custom_weights={"default": 1}, PATH_GEOJSON="../Data/Clean/Analisi/Istruzione_nopesi.geojson")

In [9]:
gdf_analisi_istruzione_nopesi = gpd.read_file("../Data/Clean/Analisi/Istruzione_nopesi.geojson", driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [10]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_istruzione_nopesi_mod = gdf_analisi_istruzione_nopesi[gdf_analisi_istruzione_nopesi["artificial"] != True]

# Leggi i due POI
gdf_scuole = gpd.read_file(PATH_SCUOLE_CLEAN)
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_scuole.geometry, gdf_biblioteche.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_istruzione_nopesi_mod.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_istruzione_nopesi_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_analisi_istruzione_nopesi_mod.to_file("../Data/Clean/Analisi/istruzione_nopesi_senza_poi.geojson", driver="GeoJSON")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_istruzione_nopesi_mod_most_highways = gdf_analisi_istruzione_nopesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [11]:
# Stampiamo il risultato
print(gdf_analisi_istruzione_nopesi_mod_most_highways)
print(f"Tot: {gdf_analisi_istruzione_nopesi_mod_most_highways["length"].sum()}")

                              length
highway                             
footway                   121.765413
residential                42.096350
unclassified               23.689185
tertiary                   19.014940
service                     9.596027
secondary                   8.968093
cycleway                    6.613318
primary                     3.950768
pedestrian                  3.414012
track                       2.148947
motorway                    1.380705
motorway_link               1.333613
footway steps               1.080161
footway service             0.975948
construction                0.656266
residential footway         0.483355
path                        0.427215
residential service         0.330508
residential pedestrian      0.268137
unclassified path           0.240288
footway pedestrian          0.232494
unclassified residential    0.220694
track service path          0.217139
tertiary_link               0.205061
residential cycleway        0.162053
t